# Implementando la app en el dispositivo
En la libreta anterior desarrollamos una aplicacion de forma iteractiva utilizando Python en el dispositivo. El verdadedo poder de ést ágil modelo de desarrollo está en que podemos probar línea por línea cada una de las ideas que deseamos implementar. En esta libreta vamos a consolidar todo el código en una sola celda y ejecutar el script desde el dispositivo, independientemente de la computadora. 


## Atencion
Para ejecutar esta libreta primero tienen que configurar su sistema como se describe en [la libreta de preparacion](./L00_Preparacion.ipynb). Comenzaremos por conectarnos al dispositivo y asegurar que podemos comunicarnos con el.

In [18]:
# necesitamos estar conectados a nuestro movil
!adb devices
!adb forward tcp:9999 tcp:2222

In [2]:
import android

In [3]:
droid = android.Android()
droid.ttsSpeak("listo para recibir instrucciones|")

Result(id=0, result=None, error=None)

## Objetivo

Implementar y ejecutar la aplicacion que habla el estado del tiempo en el dispositivo movil.

Comenzamos!

## Consolidando el código de la aplicación

Cuando ejecutan la siguiente celda, el sistema salva el codigo en un archivo con nombre `dimeElEstadoDelTiempo.py` en el directorio actual. Este archivo lo copiaremos directamente al dispositivo usando `adb` y lo ejecutaremos en el dispositivo.

In [5]:
#%%file dimeElEstadoDelTiempo.py3

# NOMBRE DEL ALUMNO

# modulos y librerias usadas
import android
import datetime
import urllib2, json
import sys
reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')

droid = android.Android()

ultimaLocalizacionConocida = droid.getLastKnownLocation().result

# asignamos una variable al contenido de la llave que no es nula usando una condicional

if ultimaLocalizacionConocida['gps'] is not None:
    ultimaLoc = ultimaLocalizacionConocida['gps']
else:
    ultimaLoc = ultimaLocalizacionConocida['network']

# del diccionario extraemos latitud y longitud y fecha
lat = ultimaLoc["latitude"]
lon = ultimaLoc["longitude"]
time = ultimaLoc["time"]

fecha = datetime.datetime.fromtimestamp(time/1e3).strftime('%Y-%m-%d %H:%M:%S')

direccion = droid.geocode(lat,lon).result

codigoPostal = direccion[0]["postal_code"]
codigoPais = direccion[0]["country_code"]

url="http://api.openweathermap.org/data/2.5/weather?zip={0},{1}&APPID=dfeea5e604e1948b28652216ad910d11".format(codigoPostal,codigoPais)

response = urllib2.urlopen(url)
respuesta = response.read()
response.close()

estadoDelTiempo = json.loads(respuesta)

# es una lista
descripcion = estadoDelTiempo["weather"][0]['description']

# mensaje
mensaje='En la ciudad de {0}, siendo las {1} horas, en la calle de {2}, el estado del tiempo es {3}'.format( \
                                direccion[0]['locality'], fecha,direccion[0]['thoroughfare'],descripcion)

droid.ttsSpeak(mensaje)

Writing dimeElEstadoDelTiempo.py3


In [7]:
# copia al dispositivo
!adb push dimeElEstadoDelTiempo.py3 /sdcard/sl4a/scripts/RySI

14 KB/s (1457 bytes in 0.095s)


La celda anterior copia el script al folder apropiado en el dispositivo; el video que se muestra a continuacion crea un nuevo folder RySI y muestra como el script se copia en esa ruta. Tambien muestra que sucede cuando ejecutamos el script directamente.

<video width="320" height="240" controls>
  <source src="../files/videos/crearNuevoFolderEnSL4A.mp4" type="video/mp4">
</video>

# Ejercicio

Modificar el programa para que 

1. el mensaje sea correcto: actualmente dice algo como "siendo las veinte de marzo de dos mil dieciocho etc." debe decir algo como "En la ciudad de XXXX siendo las HH horas con MM minutos del dia DD de AA, en la calle de CCCC...", y

2. la aplicacion muestre el mensaje al usuario a traves de una ventana de dialogo

Debido al error:

makefile() got an unexpected keyword argument 'encoding'

Fue necesario instalar python 3 desde con la apk:
https://github.com/kuri65536/python-for-android/releases/download/r32/Python3ForAndroid-debug.apk

In [16]:
%%file dimeElEstadoDelTiempo.py3             #Notar extensión de Python 3

# NOMBRE DEL ALUMNO

# modulos y librerias usadas
import android
import datetime
import urllib2, json
import sys
reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')

droid = android.Android()

ultimaLocalizacionConocida = droid.getLastKnownLocation().result

# asignamos una variable al contenido de la llave que no es nula usando una condicional

if ultimaLocalizacionConocida['gps'] is not None:
    ultimaLoc = ultimaLocalizacionConocida['gps']
else:
    ultimaLoc = ultimaLocalizacionConocida['network']

# del diccionario extraemos latitud y longitud y fecha
lat = ultimaLoc["latitude"]
lon = ultimaLoc["longitude"]
time = ultimaLoc["time"]

fecha = datetime.datetime.fromtimestamp(time/1e3).strftime('%H:%M:%S')

direccion = droid.geocode(lat,lon).result

codigoPostal = direccion[0]["postal_code"]
codigoPais = direccion[0]["country_code"]

url="http://api.openweathermap.org/data/2.5/weather?zip={0},{1}&APPID=dfeea5e604e1948b28652216ad910d11".format(codigoPostal,codigoPais)

response = urllib2.urlopen(url)
respuesta = response.read()
response.close()

estadoDelTiempo = json.loads(respuesta)

# es una lista
descripcion = estadoDelTiempo["weather"][0]['description']

# mensaje
mensaje='En la ciudad de {0}, siendo las {1} horas, en la calle de {2}, el estado del tiempo es {3}'.format( \
                                direccion[0]['locality'], fecha,direccion[0]['thoroughfare'],descripcion)

droid.dialogCreateAlert("Estado del tiempo",mensaje)
droid.dialogSetNeutralButtonText("Ok")
droid.ttsSpeak(mensaje)
droid.dialogShow()
droid.dialogGetResponse()

In [19]:
!adb push dimeElEstadoDelTiempo.py3 /sdcard0/sl4a/scripts/RySI